In [4]:
import numpy as np
import pandas as pd
import cvxopt
import os
import json

In [28]:
# Hyperparameters #
folder='10'
target_return=0.2

In [29]:
with open(os.path.join(folder, 'commandline_args.txt')) as file:
    d=json.loads(file.read())

In [30]:
preds=np.load(os.path.join(folder, 'preds_array.npy'))
data=np.array(pd.read_csv('data.csv').iloc[:, :])
assert preds.shape==data.shape

In [37]:
import cvxopt as opt
from cvxopt import blas, solvers
from cvxopt import matrix

def markowitz_no_constraint(X):
    R = np.copy(X)
    for i in range(1, R.shape[0]):
        R[i] = (X[i] - X[i-1])/X[i-1]

    R = R[1:, :]

    # Solve optimiation problem #
    R_m = R - np.mean(R, axis=0)
    omega = np.dot(R_m.T, R_m)/(np.shape(R_m)[0])

    # No target return constraint #
    e = np.ones((30, 1))
    u = np.dot(np.linalg.inv(omega), e)
    w_opt = u/(np.dot(e.T, u))
    assert np.allclose(np.sum(w_opt), 1.0), "w_opt not summing to 1.0"
    mu = np.mean(R, axis=0).reshape(-1, 1)
    
    portfolio_return = np.sum(np.dot(mu.T, w_opt))
    portfolio_risk = np.dot(w_opt.T, np.dot(omega, w_opt))[0][0]
    sharpe_ratio=portfolio_return/portfolio_risk
    
    return portfolio_return, portfolio_risk, sharpe_ratio, w_opt

def markowitz_target_return(X, target_return):
    R = np.copy(X)
    for i in range(1, R.shape[0]):
        R[i] = (X[i] - X[i-1])/X[i-1]

    R = R[1:, :]

    # Solve optimiation problem #
    R_m = R - np.mean(R, axis=0)
    omega = np.dot(R_m.T, R_m)/(np.shape(R_m)[0])
    
    """
    1/2 x'Px + q'x
    subject to :
    Gx <= h
    Ax = b

    sol = solvers.qp(P,q,G,h,A,b)
    """
    nAssets = omega.shape[0]
    e = np.ones(nAssets).reshape(1, -1)
    mu = np.mean(R, axis=0).reshape(1, -1)

    P = matrix(2.0 * omega, tc='d') # tc='d' for double matrices
    q = matrix(np.zeros(nAssets))
    G = matrix(np.zeros((nAssets, nAssets)), tc='d')
    h = matrix(np.zeros(nAssets), tc='d')
    A = matrix(np.vstack([e, mu]), tc='d')
    b = matrix([1.0, target_return])

    sol = solvers.qp(P,q,G,h,A,b)

    w_opt = np.array(sol['x'])

    assert np.allclose(np.sum(w_opt), 1.0), "w_opt not summing to 1.0"
    assert np.allclose(np.dot(mu, w_opt), target_return), "mu'w not 0.2"
    
    portfolio_return = np.sum(np.dot(mu, w_opt))
    portfolio_risk = np.dot(w_opt.T, np.dot(omega, w_opt))[0][0]
    sharpe_ratio=portfolio_return/portfolio_risk
    
    return portfolio_return, portfolio_risk, sharpe_ratio, w_opt

In [40]:
ret, risk, sr, w_opt = markowitz_no_constraint(preds)
print(sr)

7.781843489703563


In [41]:
ret, risk, sr, w_opt = markowitz_target_return(preds, target_return)
print(sr)

     pcost       dcost       gap    pres   dres
 0:  3.0077e-01  3.0077e-01  3e+01  5e+00  1e-17
 1:  3.0077e-01  3.0077e-01  3e-01  5e-02  1e-17
 2:  3.0077e-01  3.0077e-01  3e-03  5e-04  7e-18
 3:  3.0077e-01  3.0077e-01  3e-05  5e-06  9e-18
 4:  3.0077e-01  3.0077e-01  3e-07  5e-08  1e-17
Optimal solution found.
0.6649544598744258
